In [1]:
import cv2
import numpy as np
import glob
import argparse
import os

In [2]:

def detect_and_match_features(img1, img2):
    # SIFT keypoints + descriptors
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    # BFMatcher with L2 norm
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = sorted(bf.match(des1, des2), key=lambda x: x.distance)
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    return pts1, pts2

In [3]:
def ransac_homography(src_pts, dst_pts, thresh=5.0, max_iter=2000):
    best_H, best_inliers = None, []
    n = len(src_pts)
    if n < 4:
        return None
    for _ in range(max_iter):
        idx = np.random.choice(n, 4, replace=False)
        H = cv2.getPerspectiveTransform(src_pts[idx], dst_pts[idx])
        proj = cv2.perspectiveTransform(src_pts.reshape(-1,1,2), H).reshape(-1,2)
        d = np.linalg.norm(dst_pts - proj, axis=1)
        inliers = np.where(d < thresh)[0]
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_H = H
    if best_H is not None and len(best_inliers) >= 4:
        best_H, _ = cv2.findHomography(src_pts[best_inliers], dst_pts[best_inliers], 0)
    return best_H


In [4]:
def accumulate_homographies(images, ref_idx=0):
    N = len(images)
    Hs = [None]*N
    Hs[ref_idx] = np.eye(3, dtype=np.float32)
    # backward
    for i in range(ref_idx-1, -1, -1):
        p_src, p_dst = detect_and_match_features(images[i], images[i+1])
        H = ransac_homography(p_src, p_dst)
        Hs[i] = Hs[i+1] @ np.linalg.inv(H)
    # forward
    for i in range(ref_idx+1, N):
        p_src, p_dst = detect_and_match_features(images[i], images[i-1])
        H = ransac_homography(p_src, p_dst)
        Hs[i] = Hs[i-1] @ H
    return Hs


In [8]:
def warp_images(images, Hs):
    # Compute canvas bounds
    corners = []
    for img, H in zip(images, Hs):
        h, w = img.shape[:2]
        pts = np.array([[0,0],[w,0],[w,h],[0,h]], np.float32).reshape(-1,1,2)
        warped = cv2.perspectiveTransform(pts, H)
        corners.append(warped.reshape(-1,2))
    all_pts = np.vstack(corners)
    x_min, y_min = np.int32(all_pts.min(axis=0) - 0.5)
    x_max, y_max = np.int32(all_pts.max(axis=0) + 0.5)
    trans = np.array([[1,0,-x_min],[0,1,-y_min],[0,0,1]], np.float32)
    canvas = (x_max-x_min, y_max-y_min)

    warped_imgs, masks = [], []
    for img, H in zip(images, Hs):
        Ht = trans @ H
        wimg = cv2.warpPerspective(img, Ht, canvas)
        mask = cv2.warpPerspective(np.ones(img.shape[:2], np.uint8), Ht, canvas)
        warped_imgs.append(wimg)
        masks.append(mask)
    return warped_imgs, masks

In [5]:
def blend_images_poisson(warped_imgs, masks, ref_idx=0):
    # 시작 파노라마는 기준 이미지의 워핑 결과
    panorama = warped_imgs[ref_idx].copy()
    N = len(warped_imgs)
    for i in range(N):
        if i == ref_idx:
            continue
        mask = masks[i]
        if mask.sum() == 0:
            continue
        # 마스크 영역 ROI 계산
        x,y,w,h = cv2.boundingRect(mask)
        src_roi  = warped_imgs[i][y:y+h, x:x+w]
        mask_roi = mask[y:y+h, x:x+w]
        # 중심점
        center = (x + w//2, y + h//2)
        # Poisson 블렌딩
        panorama = cv2.seamlessClone(src_roi, panorama, mask_roi, center, cv2.NORMAL_CLONE)
    return panorama

In [6]:
def stitch_images(images, ref_idx=0):
    Hs = accumulate_homographies(images, ref_idx)
    warped, masks = warp_images(images, Hs)
    pano = blend_images_poisson(warped, masks, ref_idx)
    return pano

In [7]:






def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--images_dir",  required=True, help="Input images folder")
    parser.add_argument("--output",      default="panorama.jpg", help="Output filename")
    parser.add_argument("--ref_idx", type=int, default=0, help="Reference image index")
    args = parser.parse_args()

    paths = sorted(glob.glob(os.path.join(args.images_dir, "*.*")))
    imgs  = [cv2.imread(p) for p in paths]
    if any(im is None for im in imgs):
        raise RuntimeError("이미지를 불러오는 데 실패했습니다.")
    
    result = stitch_images(imgs, args.ref_idx)
    cv2.imwrite(args.output, result)
    print(f"파노라마가 {args.output}에 저장되었습니다.")




In [ ]:
if __name__ == "__main__":
    main()